# Импортирование необходимых модулей

In [1]:
import re
from typing import List

import pyspark.sql as sql

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank
from pyspark.sql.functions import col, max, sum, desc, countDistinct

# Инициализация сессии

In [2]:
spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
    .getOrCreate()

In [6]:
spark.version

'3.1.2.0-eep-800'

Перенос локальных файлов в распределенную файловую систему(HDFS)

In [18]:
# !hadoop fs -put posts__sample.xml /user/glebilin6
# !hadoop fs -put programming-languages.csv /user/glebilin6

put: `/user/glebilin6/programming-languages.csv': File exists


In [19]:
# Проверка наличия файлов в HDFS
# !hadoop fs -ls /user/glebilin6/posts__sample.xml
# !hadoop fs -ls /user/glebilin6/programming-languages.csv

-rwxr-xr-x   3 glebilin6 glebilin6      40269 2023-11-25 20:16 /user/glebilin6/programming-languages.csv


# Загрузка данных

## Указание путей с файлами датасетов

In [22]:
import os
# data_path = os.path.join(os.curdir, "data")
posts_path = os.path.join("posts__sample.xml")
prog_lang_path = os.path.join("programming-languages.csv")

## Чтение данных о упоминаемости языков программирования в постах на Stack Overflow

In [16]:
posts_data = spark.read.format('xml').options(rowTag='row').load(posts_path)

print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



## Чтение данных о языках программирования

In [24]:
prog_lang_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv(prog_lang_path)

print("Programming languages")
prog_lang_data.printSchema()

Programming languages
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



# Работа с данными

## Функции для обработки содержимого столбцов с тегами и датой последней активности поста

In [27]:
def get_tags(tags_string):
    if tags_string is None:
        return []
    
    pattern = r'<(.+?)>'
    tags = re.findall(pattern, tags_string)
    
    return tags

def get_year(date_and_time):
    return date_and_time.year

get_tags_udf = udf(get_tags, ArrayType(StringType()))
get_year_udf = udf(get_year, IntegerType())

## Выбор необходимых столбцов

In [28]:
posts_data_simplified = posts_data \
                        .withColumn("tags", get_tags_udf(posts_data["_Tags"])) \
                        .withColumn("year", get_year_udf(posts_data["_LastActivityDate"]))
posts_data_simplified = posts_data_simplified.select(
    col("tags"),
    col("year"),
    col("_ViewCount").alias("views")
)

first_rows = posts_data_simplified.head(10)
for i, row in enumerate(first_rows):
    print(i+1, row)

1 Row(tags=['c#', 'floating-point', 'type-conversion', 'double', 'decimal'], year=2019, views=42817)
2 Row(tags=['html', 'css', 'internet-explorer-7'], year=2019, views=18214)
3 Row(tags=[], year=2017, views=None)
4 Row(tags=['c#', '.net', 'datetime'], year=2019, views=555183)
5 Row(tags=['c#', 'datetime', 'time', 'datediff', 'relative-time-span'], year=2019, views=149445)
6 Row(tags=[], year=2018, views=None)
7 Row(tags=['html', 'browser', 'timezone', 'user-agent', 'timezone-offset'], year=2019, views=176405)
8 Row(tags=['.net', 'math'], year=2018, views=123231)
9 Row(tags=[], year=2010, views=None)
10 Row(tags=[], year=2010, views=None)


## Составление полного отчета

In [29]:
# Разбиение массива тегов на отдельные столбцы
posts_data_sorted = posts_data_simplified.select("year", explode("tags").alias("tag"), "views")

# Группировка по году последней активности и тегам, суммирование всех просмотров для каждого языка программирования в пределах одного года
posts_data_sorted = posts_data_sorted.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Сортировка по году и количеству просмотров
posts_data_sorted = posts_data_sorted.orderBy("year", desc("total_views"))

# Отображение отчета
posts_data_sorted.show()

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|            database|      19682|
|2008|               local|      19682|
|2008|                java|      11532|
|2008|         inheritance|      10971|
|2008|       accessibility|       7700|
|2008|           variables|       7700|
|2008|               excel|       6540|
|2008|          automation|       6540|
|2008|           interface|       3271|
|2008|      castle-windsor|       2927|
|2008|dependency-injection|       2927|
|2008|       configuration|       2927|
|2008|               linux|       2393|
|2008|       ruby-on-rails|       1843|
|2008|                ruby|       1843|
|2008|            .net-3.0|       1432|
|2008|  visual-studio-2008|       1432|
|2008|  visual-studio-2005|       1432|
+----+--------------------+-----------+
only showing top 20 rows



## Составление итогового отчета

In [30]:
#  Определяем спецификацию Window, разбитую по годам и упорядоченную в порядке убывания по показателю total_views
window_spec = Window.partitionBy("year").orderBy(posts_data_sorted["total_views"].desc())

# Добавляем колонку rank в DataFrame
ranked_df = posts_data_sorted.withColumn("rank", rank().over(window_spec))

# Отбираем первые 5 строк (5 наиболее популярных языков программирования в определенный год)
result_df = ranked_df.filter(ranked_df["rank"] <= 5)

# Выбирам колоки год, язык программирования и общее число просмотров
result_df = result_df.select("year", "tag", "total_views")

# Представляем результат по топ 5 самым популярным языкам программирования на StackOverflow с 2008 года
posts_data_sorted_result = result_df.orderBy("year", desc("total_views"))

# Show the final DataFrame
posts_data_sorted_result.show()

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|            database|      19682|
|2008|               local|      19682|
|2008|                java|      11532|
|2009|                  c#|      73661|
|2009|                .net|      39167|
|2009|              python|      32219|
|2009|                 c++|      29381|
|2009|            winforms|      25670|
|2010|                  c#|     128597|
|2010|              arrays|      80868|
|2010|                java|      53333|
|2010|              matlab|      51865|
|2010|multidimensional-...|      51865|
|2011|                  c#|     238076|
|2011|                java|     121315|
|2011|                .net|     120734|
|2011|                 css|     119302|
|2011|             android|     107283|
+----+--------------------+-----------+
only showing top 20 rows



In [37]:
#Сохраняем данные в формат Parquet
import tempfile
with tempfile.TemporaryDirectory() as d:
    # Write a DataFrame into a Parquet file
    posts_data_sorted_result.write.parquet(d)

    